In [1]:
"""
SPDX-License-Identifier: GPL-3.0-or-later
© 2008-2025 San Diego State University Research Foundation (SDSURF).
See LICENSE file or https://www.gnu.org/licenses/gpl-3.0.html for details. 

@file gradient.cpp
@brief Mimetic Gradient Operator Class
@date 2025/06/01
"""

'\nSPDX-License-Identifier: GPL-3.0-or-later\n© 2008-2025 San Diego State University Research Foundation (SDSURF).\nSee LICENSE file or https://www.gnu.org/licenses/gpl-3.0.html for details. \n\n@file gradient.cpp\n@brief Mimetic Gradient Operator Class\n@date 2025/06/01\n'

In [2]:
import numpy as np
from scipy.sparse import csr_array

In [65]:
class Gradient:
    def __init__(self, k, m, dx):

        assert (k % 2) == 0, "k must be even"
        assert 1 < k < 9, "valid range of k is [1,9]"
        assert m >= 2 * k, "m must be greater or equal to 2k"

        self.k =  k
        self.m = m
        self.dx = dx
        
        match k:
            case 2:
                data = np.array([
                    -8.0 / 3.0, 3.0, -1.0 / 3.0, 
                    1.0 / 3.0, -3.0, 8.0 / 3.0])
                row = np.array([0, 0, 0, m, m, m])
                col = np.array([0, 1, 2, m-1, m, m+1])
            
                data = np.insert(data, 3, np.ones(m-1))
                row = np.insert(row, 3, np.array(range(1,m)))
                col = np.insert(col, 3, np.array(range(1,m)))

                self.G = csr_array((data, (row, col)), dtype=np.float64)
                # Weights
                P = { 3.0 / 8.0 , 9.0 / 8.0 , 1.0 , 9.0 / 8.0 , 3.0 / 8.0 }

            case 4:
                row0 = np.array([-352.0 / 105.0, 35.0 / 8.0, -35.0 / 24.0, 21.0 / 40.0, -5.0 / 56.0])
                row1 = np.array([ 16.0 / 105.0, -31.0 / 24.0, 29.0 / 24.0, -3.0 / 40.0, 1.0 / 168.0])
                data = np.concatenate([
                    row0,
                    row1,
                    -1.0 * row1[::-1],
                    -1.0 * row0[::-1]                    
                    ])
                row = np.concatenate([
                    np.zeros(k+1), 
                    np.full(k+1, 1),
                    np.full(k+1, m-1),
                    np.full(k+1, m)
                    ]).astype(int)
                col = np.concatenate([
                    np.arange(k+1),
                    np.arange(k+1), 
                    np.arange(m-3,m+2),
                    np.arange(m-3,m+2)
                    ])
                DATAM =  np.array([1.0 / 24.0, -9.0 / 8.0, 9.0 / 8.0, -1.0 / 24.0])
                dataM = np.array([])
                rowM = np.array([])
                colM = np.array([])
                for i in range(2, m - 1):
                    rowM = np.append(rowM, np.full(4,i)).astype(int)
                    colM = np.append(colM, np.arange(i-1,i+3)).astype(int)
                    dataM = np.append(dataM, DATAM)

                offset = len(row0)+len(row1)
                data = np.insert(data, offset, dataM)
                row = np.insert(row, offset, rowM)
                col = np.insert(col, offset, colM)

                self.G = csr_array((data, (row, col)), dtype=np.float64)

    def toarray(self):
        return self.G.toarray()



In [66]:
k = 2
m = 2*k+1
dx = 1/m
G = Gradient(k, m, dx)
np.set_printoptions(linewidth=160) 
G.toarray()

array([[-2.66666667,  3.        , -0.33333333,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.33333333, -3.        ,  2.66666667]])

In [67]:
k = 4
m = 2*k+1
dx = 1/m
G = Gradient(k, m, dx)
np.set_printoptions(linewidth=160) 
G.toarray()

array([[-3.35238095,  4.375     , -1.45833333,  0.525     , -0.08928571,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.15238095, -1.29166667,  1.20833333, -0.075     ,  0.00595238,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.04166667, -1.125     ,  1.125     , -0.04166667,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.04166667, -1.125     ,  1.125     , -0.04166667,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.04166667, -1.125     ,  1.125     , -0.04166667,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.04166667, -1.125     ,  1.125     , -0.04166667,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.04166667, -1